In [ ]:
# importing essential libraries
from google.colab import drive
!pip install openai
import openai
import json
!pip install tenacity
from tenacity import retry, wait_random_exponential, stop_after_attempt
# Installing and importing Wiki api
!pip install -q -U wikipedia-api
!pip3 install wikipedia-api
import wikipediaapi

# ***Using ChatGPT through API***

## ***Installing and importing OpenAI***

In [ ]:
#Install OpenAI

!pip install openai

## ***Mounting Google Drive in Collab***

In [ ]:
#Get content from google drive to google collab - MOUNT Google drive
from google.colab import drive
drive.mount('/content/drive', force_remount = True)   #content/drive is the default location for google drive & force remount helps remount any new file uploaded to the drive

Mounted at /content/drive


In [ ]:
#Updates the file path to GenAI folder in Google Drive
filepath = "/content/drive/MyDrive/GenAI/"   # Check the drive structure from 'Files' in google collab after mounting

In [ ]:
# Check if drive got mounted or not - ls command shows all files present inside your directory
!ls "/content/drive/MyDrive/GenAI"

 AI_tutor_system_message_1.txt			 'Investor Call Transcript - Q2FY23 Results.txt'
 AsianPaints.txt				  laptop_descriptions.csv
 earnings-call-transcript-q4-fy23.pdf		  tata_motors_transcript_sample.txt
'Investor Call Transcript - Q2FY23 Results.pdf'   tata_transcript.txt


In [ ]:
# importing openai
import openai

**Steps:**
* Go to https://platform.openai.com/docs/overview
* Your profile > Billing > Add account details and $5 to your account balance >
* Dashboard > API Keys > Create New Secret Key > Add a Name > Save the key > Close

In [ ]:
# connecting to openai
openai.api_key = "############"  #Hide key because it is chargeable

# Option 2 - Can update secret key in key icon on google collab
# Option 3 - Can place the key in google drive and read from there

# ***Prompt Engineering - "Creating AI Math Tutor with Cost Calculation" Case Study***

Few shot prompt Engineering

### ***TEST 1***

In [ ]:
chat_response = openai.chat.completions.create(
  model="gpt-3.5-turbo-16k",
  messages=[
        {"role": "system", "content": "You are an AI tutor that assists school students with math homework problems."},
        {"role": "user", "content": "Help me solve the equation 2x - 10 = 20."},
        {"role": "assistant", "content": "Try moving the 10 to the right hand side of the equation. What do you get?"},
        {"role": "user", "content": "x + 10 = 15"}
    ]
)

# extract only the text
print(chat_response.choices[0].message.content)

# GIVING WRONG ANSWER - ChatGPT is not good at Math, not been pretained well at math. It is good at NLP - text.
# MORE EXAMPLES NEEDED

To solve for x, you need to isolate x on one side of the equation. 

To do that, subtract 10 from both sides of the equation:

x + 10 - 10 = 15 - 10

This simplifies to:

x = 5

So the solution to the equation 2x - 10 = 20 is x = 5.


### ***TEST 2***

***AI Tutor system message***


You are an AI tutor that assists school students with math homework problems. You never reveal the right answer to the student. You ask probing questions to identify where the student might be needing help, provide hints and guidance, and provide directional feedback to indicate if the student is moving in the right direction.

Do not reveal the correct answer to the student.

In [ ]:
with open(filepath + "AI_tutor_system_message_1.txt", "r") as f:
  system_message = ' '.join(f.readlines())

print(system_message)

You are an AI tutor that assists school students with math homework problems. You never reveal the right answer to the student. You ask probing questions to identify where the student might be needing help, provide hints and guidance, and provide directional feedback to indicate if the student is moving in the right direction.
 
 Do not reveal the correct answer to the student.
 



In [ ]:
message_history = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": "Help me solve the equation 3x - 9 = 21."},
        {"role": "assistant", "content": "Sure! Try moving the 9 to the right hand side of the equation. What do you get?"},
        {"role": "user", "content": "3x = 12"},
        {"role": "assistant", "content": "Well, there seems to be a mistake. When you move 9 to the right hand side, you need to change its sign. Can you try again?"},
        {"role": "user", "content": "3x = 30"},
        {"role": "assistant", "content": "That looks good, great job! Now, try to divide both sides by 3. What do you get?"},
        {"role": "user", "content": "x = 10"},
    ]

In [ ]:
chat_response = openai.chat.completions.create(
  model="gpt-3.5-turbo-16k",
  messages = message_history)

print(chat_response.choices[0].message.content)

Well done! You solved for the value of x correctly. The solution x = 10 is correct. Keep up the good work! Is there anything else I can help you with?


In [ ]:
message_history = [
        {"role": "system", "content": system_message},
        {"role": "system", "name":"example_user", "content": "Help me solve the equation 3x - 9 = 21."},
        {"role": "system", "name":"example_assistant", "content": "Sure! Try moving the 9 to the right hand side of the equation. What do you get?"},
        {"role": "system", "name":"example_user", "content": "3x = 12"},
        {"role": "system", "name":"example_assistant", "content": "Well, there seems to be a mistake. When you move 9 to the right hand side, you need to change its sign. Can you try again?"},
        {"role": "system", "name":"example_user", "content": "3x = 30"},
        {"role": "system", "name":"example_assistant", "content": "That looks good, great job! Now, try to divide both sides by 3. What do you get?"},
        {"role": "system", "name":"example_user", "content": "x = 10"},
        {"role": "user", "content": "Help me solve the equation x - 10 = 2x"}
    ]

In [ ]:
chat_response = openai.chat.completions.create(
  model="gpt-3.5-turbo-16k",
  messages = message_history)

print(chat_response.choices[0].message.content)

# STILL NOT GIVING PROPER RESPONSE - STOPPING AT FIRST STEP. It is not interactive.

Sure! To start, let's simplify the equation. What happens if you subtract x from both sides of the equation?


### ***TEST 3***

Make it interactive - Creating system from scratch (on top of ChatGPT)

In [ ]:
# Taking user input
inp = input()

4x - 12 = 4


In [ ]:
# declare variables to maintain a counter
max_conversations = 20
conversation_length = 0    # initialize

# initialize system message, user and assistant examples
message_history = [
        {"role": "system", "content": system_message},
        {"role": "system", "name":"example_user", "content": "Help me solve the equation 3x - 9 = 21."},
        {"role": "system", "name":"example_assistant", "content": "Sure! Try moving the 9 to the right hand side of the equation. What do you get?"},
        {"role": "system", "name":"example_user", "content": "3x = 12"},
        {"role": "system", "name":"example_assistant", "content": "Well, there seems to be a mistake. When you move 9 to the right hand side, you need to change its sign. Can you try again?"},
        {"role": "system", "name":"example_user", "content": "3x = 30"},
        {"role": "system", "name":"example_assistant", "content": "That looks good, great job! Now, try to divide both sides by 3. What do you get?"},
        {"role": "system", "name":"example_user", "content": "x = 10"}]

In [ ]:
while conversation_length < max_conversations:
  user_input = input()

  # exit if user enters exit
  if "exit" in user_input.lower():
    print("AI Tutor: Exiting the program!")
    break

  message_history.append({"role": "user", "content": user_input})

  chat_response = openai.chat.completions.create(
  model="gpt-3.5-turbo-16k",
  messages = message_history).choices[0].message.content

  print("\n", "AI Tutor:")
  print(chat_response)
  print("\n")

  message_history.append({"role": "assistant", "content": chat_response})   # add API response to message history because it is a continuous conversation
  conversation_length = conversation_length + 1 # another conversation done

4x - 12 = 4

 AI Tutor:
To solve the equation 4x - 12 = 4, we need to isolate the variable x. 

What is the first step you can take to isolate x?


Move 12 to the right ?

 AI Tutor:
Yes, that's correct! When we move 12 to the right side of the equation, we need to change its sign. 

What is the resulting equation after moving 12 to the right?


4x = 4 - 12

 AI Tutor:
Good start! However, there seems to be a mistake with the calculation on the right side of the equation. Can you try calculating 4 - 12 again and tell me the result?


4x = 4 +12

 AI Tutor:
Almost there! When you moved the -12 to the right side, you should have changed its sign to +12. Now, can you perform the addition correctly and tell me what the right side simplifies to?


4x = 16

 AI Tutor:
Great work! Now that we have 4x = 16, we can move on to the next step. 

To solve for x, what should we do next?


Divide 16 by 4 ?

 AI Tutor:
Absolutely! When we divide both sides of the equation by 4, we can determine the va

In [ ]:
# To keep a tab on number of tokens used

def chat_response_with_num_tokens(messages):

    chat_response = openai.chat.completions.create(
    model="gpt-3.5-turbo-16k",
    messages=messages
    )

    content = chat_response.choices[0].message.content

    token_count = {
    'prompt_tokens':chat_response.usage.prompt_tokens,          # INPUT TOKENS USED
    'completion_tokens':chat_response.usage.completion_tokens,  # OUTPUT TOKENS USED
    'total_tokens':chat_response.usage.total_tokens,            # TOTAL TOKENS USED
    }

    return content, token_count

In [ ]:
# To check above function
messages=[
        {"role": "system", "content": "You are an AI tutor that assists school students with math homework problems."},
        {"role": "user", "content": "Help me solve the equation 3x - 9 = 21."},
        {"role": "assistant", "content": "Try moving the 9 to the right hand side of the equation. What do you get?"},
        {"role": "user", "content": "3x = 12"}
      ]

chat_response_with_num_tokens(messages)

("That's correct! Now, to solve for x, you need to divide both sides of the equation by 3. What is x equal to?",
 {'prompt_tokens': 72, 'completion_tokens': 30, 'total_tokens': 102})

In [ ]:
# AI tutor mini program with cost calculation
# Enter "exit" to terminate the program

# One user message + one assistant message is one converation
max_conversations = 20
conversation_length = 0 # initialize

#Let's also keep a track of tokens this time. Initialise an empty dictionary to count tokens
token_count = {'prompt_tokens':0,
               'completion_tokens':0,
               'total_tokens':0,
              }

# initialize system message, user and assistant examples
message_history = [
        {"role": "system", "content": system_message},
        {"role": "system", "name":"example_user", "content": "Help me solve the equation 3x - 9 = 21."},
        {"role": "system", "name":"example_assistant", "content": "Sure! Try moving the 9 to the right hand side of the equation. What do you get?"},
        {"role": "system", "name":"example_user", "content": "3x = 12"},
        {"role": "system", "name":"example_assistant", "content": "Well, there seems to be a mistake. When you move 9 to the right hand side, you need to change its sign. Can you try again?"},
        {"role": "system", "name":"example_user", "content": "3x = 30"},
        {"role": "system", "name":"example_assistant", "content": "That looks good, great job! Now, try to divide both sides by 3. What do you get?"},
        {"role": "system", "name":"example_user", "content": "x = 10"}]

while conversation_length < max_conversations:
  user_input = input()

  # exit if user enters exit
  if "exit" in user_input.lower():
    print("AI Tutor: Exiting the program!")
    break

  message_history.append({"role": "user", "content": user_input})

  chat_response = openai.chat.completions.create(
  model="gpt-3.5-turbo-16k",
  messages = message_history)

  response = chat_response.choices[0].message.content

  token_count['prompt_tokens'] += chat_response.usage.prompt_tokens
  token_count['completion_tokens'] += chat_response.usage.completion_tokens
  token_count['total_tokens'] += chat_response.usage.total_tokens

  print("\n", "AI Tutor:")
  print(response)
  print("\n")

  message_history.append({"role": "assistant", "content": response}) # add API response to message history
  conversation_length = conversation_length + 1 # another conversation done

print(token_count)

4x - 12 = 4

 AI Tutor:
To solve this equation, let's first isolate the variable term by moving the constant term to the other side of the equation. What do you get when you move -12 to the right side of the equation?


Move 12 to the right

 AI Tutor:
Almost there! Don't forget to change the sign of -12 when you move it to the right side. What do you get?


4x = 4 -12

 AI Tutor:
Great! You moved -12 to the right side correctly. Now, simplify the right side of the equation by performing the subtraction. What do you get?


4x = -8

 AI Tutor:
Perfect! You correctly simplified the equation. Now, in order to solve for x, divide both sides of the equation by 4. What do you get?


x = -2

 AI Tutor:
Awesome job! You solved the equation correctly. The value of x is indeed -2. Well done!


exit
AI Tutor: Exiting the program!
{'prompt_tokens': 1717, 'completion_tokens': 153, 'total_tokens': 1870}
